In [13]:
import modelopt.torch.quantization as mtq
from tensorrt_llm.models.llama.convert import load_calib_dataset
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import copy
from datasets import load_dataset

calib_dataset = "ccdv/cnn_dailymail"
dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0")["train"]
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
# model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")
# model.to("cuda")

for i in tqdm(range(5), desc="calibrating model"):
    datapoint = dataset[i]
    line = copy.copy(datapoint)
    # print(line)
    article = line["article"][:500] + " TL;DR: ".strip()
    article = article.strip().replace(" n't", "n't")

    highlights = line["highlights"][:500].strip().replace(" n't", "n't")
    input_ids = tokenizer(
        article,
        return_tensors="pt",
        max_length=256,
        padding=True,
        truncation=True,
    ).input_ids.to("cuda")
    labels = tokenizer(
        highlights,
        return_tensors="pt",
        max_length=256,
        padding=True,
        truncation=True,
    ).input_ids.to("cuda")

    model(
        input_ids=input_ids,
        labels=labels,
    )

/home/user-name-goes-here/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
/home/user-name-goes-here/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
calibrating model: 100%|██████████| 5/5 [00:00<00:00,  5.23it/s]


In [7]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file


tensors = {}
with safe_open(
    "spellchecker/kosenko/libs/TensorRT_LLM/examples/enc_dec/tmp/trt_models/google-t5/t5-small/bfloat16/tp1/pp1/decoder/rank0.safetensors",
    framework="pt",
    device="cpu",
) as f:
    for key in f.keys():
        print(key)
        tensors[key] = f.get_tensor(key)

decoder_layers.0.mlp.fc.prequant_scaling_factor
decoder_layers.0.mlp.fc.weight
decoder_layers.0.mlp.fc.weights_scaling_factor
decoder_layers.0.mlp.proj.prequant_scaling_factor
decoder_layers.0.mlp.proj.weight
decoder_layers.0.mlp.proj.weights_scaling_factor
decoder_layers.0.mlp_layernorm.weight
decoder_layers.0.self_attention.dense.prequant_scaling_factor
decoder_layers.0.self_attention.dense.weight
decoder_layers.0.self_attention.dense.weights_scaling_factor
decoder_layers.0.self_attention.rel_attn_table
decoder_layers.0.self_attention_layernorm.weight
decoder_layers.1.mlp.fc.prequant_scaling_factor
decoder_layers.1.mlp.fc.weight
decoder_layers.1.mlp.fc.weights_scaling_factor
decoder_layers.1.mlp.proj.prequant_scaling_factor
decoder_layers.1.mlp.proj.weight
decoder_layers.1.mlp.proj.weights_scaling_factor
decoder_layers.1.mlp_layernorm.weight
decoder_layers.1.self_attention.dense.prequant_scaling_factor
decoder_layers.1.self_attention.dense.weight
decoder_layers.1.self_attention.dense